<a href="https://colab.research.google.com/github/bjmcnabb/DMS_Climatology/blob/main/Southern%20Ocean/Satellite_data_processing_SO_35gb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Created on Tue Oct 27 20:19:13 2020

@author: Brandon McNabb (bmcnabb@eoas.ubc.ca)
"""

'\nCreated on Tue Oct 27 20:19:13 2020\n\n@author: Brandon McNabb (bmcnabb@eoas.ubc.ca)\n'

In [ ]:
! pip install dask[dataframe]
! pip install obspy
! pip install pyhdf
! pip install numpy --upgrade

     |████████████████████████████████| 132 kB 12.4 MB/s 
     |████████████████████████████████| 24.7 MB 1.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for obspy: filename=obspy-1.2.2-cp37-cp37m-linux_x86_64.whl size=21668551 sha256=06510e31a26667b00f5a3b916b9df9ded698be34fb53593f65678db93a7d0387
  Stored in directory: /root/.cache/pip/wheels/28/7e/ea/0a37d5f5001d096cf97d6527b60300badd2d0074449e89c736
Successfully built obspy
     |████████████████████████████████| 757 kB 12.0 MB/s 
  Using cached numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# !unzip drive/MyDrive/SO_DMS_predictors.zip
!unzip drive/MyDrive/CCMP_0.25_wind_speeds.zip
# !unzip drive/MyDrive/WOA18.zip
!unzip drive/MyDrive/CERSAT-GLO-REP_WIND_L4-OBS_FULL_TIME_SERIE_1607246388558.zip

Mounted at /content/drive
Archive:  drive/MyDrive/CCMP_0.25_wind_speeds.zip
   creating: CCMP_0.25_wind_speeds/
 extracting: CCMP_0.25_wind_speeds/month_19971201_v11l35flk.nc.gz  
 extracting: CCMP_0.25_wind_speeds/month_20011201_v11l35flk.nc.gz  
 extracting: CCMP_0.25_wind_speeds/month_20020101_v11l35flk.nc.gz  
 extracting: CCMP_0.25_wind_speeds/month_20020201_v11l35flk.nc.gz  
 extracting: CCMP_0.25_wind_speeds/month_20030101_v11l35flk.nc.gz  
 extracting: CCMP_0.25_wind_speeds/month_20030201_v11l35flk.nc.gz  
  inflating: CCMP_0.25_wind_speeds/month_20030301_v11l35flk.nc  
  inflating: CCMP_0.25_wind_speeds/month_20031001_v11l35flk.nc  
 extracting: CCMP_0.25_wind_speeds/month_20031101_v11l35flk.nc.gz  
 extracting: CCMP_0.25_wind_speeds/month_20040101_v11l35flk.nc.gz  
 extracting: CCMP_0.25_wind_speeds/month_20040201_v11l35flk.nc.gz  
 extracting: CCMP_0.25_wind_speeds/month_20041201_v11l35flk.nc.gz  
 extracting: CCMP_0.25_wind_speeds/month_20050101_v11l35flk.nc.gz  
 extractin

# Process Data

In [ ]:
import timeit
runtime_start = timeit.default_timer() # start the clock

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy
import scipy.interpolate
import os
from dateutil import parser
import xarray as xr
import dask.array as da
import dask.dataframe as dd
from pyhdf.SD import SD, SDC
import datetime
from obspy.geodetics import kilometers2degrees

In [ ]:
#~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
# *** Make sure lat/lons bounds are the same as analysis script!! ***
#~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

#### Spatial grid resolution (degrees):
grid = kilometers2degrees(20) # convert km to deg

#### Define lat/lon constraints

# set spatial bounding limits
min_lon, max_lon, min_lat, max_lat = -180, 180, -90, -40

#### Define bins
latbins = np.arange(min_lat,max_lat+grid,grid)
lonbins = np.arange(min_lon,max_lon+grid,grid)

#### Set a call for which dataset is being processed
which_dataset = 'DMS'
# which_dataset = 'Fe'
if which_dataset == 'DMS':
  write_to = 'SO_DMS_'
else:
  write_to = 'SO_Fe_'

In [ ]:
start = timeit.default_timer() # start the clock
#-----------------------------------------------------------------------------
# Set path:
# file directory shape:
# -/MODIS
#       |-/Data
#           |-/1_Chl
#               |-/*.nc
#           |-/2_PIC
#               |-/*.nc
#           |-/3_SST
#               |-/*.nc
#           |-/4_PAR
#               |-/*.nc
#           |-/5_Kd
#               |-/*.nc
#           |-/6_POC
#               |-/*.nc
#           |-/7_FLH
#               |-/*.nc
#           |-/8_SSHA
#               |-/*.nc
#           |-/9_NPP
#               |-/*.hdf
#           |-/10_MIMOC_MLD
#               |-/*.hdf
#           |-/11_MIMOC_TS
#               |-/*.hdf

# Set directory to write data files to:
write_dir = 'C:/Users/bcamc/OneDrive/Desktop/Python/Projects/Sulfur/Southern Ocean/SO_DMS_data_longer_ts'
# Call directory with raw data files to process
# directory = 'E:/Satellite/SO_DMS_predictors' # E: or C:, depending if using hard drive/thumb drive
directory = 'SO_DMS_predictors/' # E: or C:, depending if using hard drive/thumb drive
folders = sorted(os.listdir(directory)) # calls specific datasets grouped in folders
num_folders = len(folders) # set this to the number of variables
#-----------------------------------------------------------------------------

print('finding dimensions...')

# ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
# preallocate variable:
dims = np.empty((num_folders,4))
# ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
for count, folder in enumerate(folders):
    path = os.path.join(directory, folder)
    for subdir, dirs, files in os.walk(path):
        for n, file in enumerate(files):
            if count < 9 and n == 1:
                if count == 0 and n == 1:
                    # folowing pulls the number of unique months within the directory files; used to estimate total data length for binning
                    num_months = np.size(np.unique([pd.to_datetime(datetime.datetime.strptime(FILE.split('.')[0][1:8], '%Y%j').date()).strftime('%m') for FILE in files]))
                # find lat/lon dimensions:
                # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
                # Open up one of the datasets
                filepath = subdir + os.sep + file
                check = xr.open_dataset(filepath)
                # Pull the lat/lon from it:
                lat = check.coords['lat'].values
                lon = check.coords['lon'].values
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                # Convert time/lats/lons into repeating matrices to match data dimensions
                lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                # Need these as pandas dataframes to using binning scheme:
                # NOTE: dummy data added to 'datetime' & 'data' columns to perserve shape 
                d = {'datetime':np.ravel(lat_mat),'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(lat_mat)}
                data_long = pd.DataFrame(data=d)
                # Bin data as averages across gridded spatial bins:
                to_bin = lambda x: np.round(x / grid) * grid
                data_long['latbins'] = data_long.lat.map(to_bin)
                data_long['lonbins'] = data_long.lon.map(to_bin)
                data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                # Rename binned columns + drop mean lat/lons:
                data_proc = data_proc.drop(columns=['lat', 'lon'])
                data_proc.reset_index(inplace=True) # remove index specification on columns
                check.close()
                del check
            if count == 9 and n == 1: # Data configured differently for SSHA data
                # folowing pulls the number of unique months within the directory files; used to estimate total data length for binning
                num_months = np.size(np.unique([pd.to_datetime(datetime.datetime.strptime(FILE.split('_')[3][:8], '%Y%m%d').date()).strftime('%m') for FILE in files]))
                # find lat/lon dimensions:
                # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
                # Open up one of the datasets
                filepath = subdir + os.sep + file
                check = xr.open_dataset(filepath)
                # Pull the lat/lon from it:
                lat = check.coords['Latitude'].values
                lon = check.coords['Longitude'].values
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                # Convert time/lats/lons into repeating matrices to match data dimensions
                lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                # Need these as pandas dataframes to using binning scheme:
                # NOTE: dummy data added to 'datetime' & 'data' columns to perserve shape 
                d = {'datetime':np.ravel(lat_mat),'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(lat_mat)}
                data_long = pd.DataFrame(data=d)
                # Bin data as averages across gridded spatial bins:
                to_bin = lambda x: np.round(x / grid) * grid
                data_long['latbins'] = data_long.lat.map(to_bin)
                data_long['lonbins'] = data_long.lon.map(to_bin)
                data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                # Rename binned columns + drop mean lat/lons:
                data_proc = data_proc.drop(columns=['lat', 'lon'])
                data_proc.reset_index(inplace=True) # remove index specification on columns
                check.close()
                del check
            if count == 10 and n == 1: # Data configured differently for NPP data
                # folowing pulls the number of unique months within the directory files; used to estimate total data length for binning
                num_months = np.size(np.unique([pd.to_datetime(datetime.datetime.strptime(FILE.split('.')[1][:7], '%Y%j').date()).strftime('%m') for FILE in files]))
                # find lat/lon dimensions:
                # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
                # Open up one of the datasets
                filepath_NPP = subdir + os.sep + file
                vars_ = SD(filepath_NPP, SDC.READ)
                # data = np.flipud(vars_.select('npp').get())
                # lets pull the lat/lon from it:
                n = 180/1080
                lat = np.arange(-90,90,n)
                lon = np.arange(-180,180,n)
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                # Convert time/lats/lons into repeating matrices to match data dimensions
                lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                # Need these as pandas dataframes to using binning scheme:
                # NOTE: dummy data added to 'datetime' & 'data' columns to perserve shape 
                d = {'datetime':np.ravel(lat_mat),'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(lat_mat)}
                data_long = pd.DataFrame(data=d)
                # Bin data as averages across gridded spatial bins:
                to_bin = lambda x: np.round(x / grid) * grid
                data_long['latbins'] = data_long.lat.map(to_bin)
                data_long['lonbins'] = data_long.lon.map(to_bin)
                data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                # Rename binned columns + drop mean lat/lons:
                data_proc = data_proc.drop(columns=['lat', 'lon'])
                data_proc.reset_index(inplace=True) # remove index specification on columns
                del vars_
            if count == 11 or count == 11 and n == 1: # Data configured differently for MLD data
                # folowing pulls the number of unique months within the directory files; used to estimate total data length for binning
                num_months = np.size([FILE.split("month")[1].split('.nc')[0] for FILE in files])
                # find lat/lon dimensions:
                # ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
                # Open up one of the datasets
                filepath = subdir + os.sep + file
                check = xr.open_dataset(filepath)
                # lets pull the lat/lon from it:
                lat = check.LATITUDE.values
                lon = check.LONGITUDE.values-180 # convert to W/E coords
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                # Convert time/lats/lons into repeating matrices to match data dimensions
                lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                # Need these as pandas dataframes to using binning scheme:
                # NOTE: dummy data added to 'datetime' & 'data' columns to perserve shape 
                d = {'datetime':np.ravel(lat_mat),'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(lat_mat)}
                data_long = pd.DataFrame(data=d)
                # Bin data as averages across gridded spatial bins:
                to_bin = lambda x: np.round(x / grid) * grid
                data_long['latbins'] = data_long.lat.map(to_bin)
                data_long['lonbins'] = data_long.lon.map(to_bin)
                data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                # Rename binned columns + drop mean lat/lons:
                data_proc = data_proc.drop(columns=['lat', 'lon'])
                data_proc.reset_index(inplace=True) # remove index specification on columns
                check.close()
                del check
            else:
                pass
    # extract number of files (datasets) per folder + lat/lon dimensions:
    dims[count] = np.array([int(count), int(np.size(files)), int(data_proc.shape[0]*num_months), int(data_proc.shape[1])])
    print('\nData length:',str(data_proc.shape[0]))
    print('In folder ' + '"' + folder + '":')
    print('Number of datasets = '+ str(np.size(files)))
    print('Number of unique months = '+ str(num_months))
end = timeit.default_timer() # stop the clock
print('\n~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~')
print('Execution time:')
print(str(round(end-start,5)),'secs')
print(str(round((end-start)/60,5)),'mins')
print(str(round((end-start)/3600,5)),'hrs')

finding dimensions...

Data length: 2403600
In folder "1_Chl":
Number of datasets = 36
Number of unique months = 7

Data length: 2403600
In folder "2_PIC":
Number of datasets = 36
Number of unique months = 7

Data length: 2403600
In folder "3_SST":
Number of datasets = 36
Number of unique months = 7

Data length: 1201800
In folder "4_PAR":
Number of datasets = 36
Number of unique months = 7

Data length: 2403600
In folder "5_Kd":
Number of datasets = 36
Number of unique months = 7

Data length: 2403600
In folder "6_POC":
Number of datasets = 36
Number of unique months = 7

Data length: 2403600
In folder "7_nFLH":
Number of datasets = 36
Number of unique months = 7

Data length: 2403600
In folder "8_iPAR":
Number of datasets = 36
Number of unique months = 7

Data length: 2403600
In folder "91_CDOM":
Number of datasets = 36
Number of unique months = 7

Data length: 240240
In folder "92_SSHA":
Number of datasets = 217
Number of unique months = 7

Data length: 600600
In folder "93_NPP":
Nu

In [ ]:
#-----------------------------------------------------------------------------
# Pre-allocate extraction variables as a dict using Dask:
variables = {'chl':dd.from_array(da.empty((dims[0,2],dims[0,3]))).compute(),
             'PIC':dd.from_array(da.empty((dims[1,2],dims[1,3]))).compute(),
             'SST':dd.from_array(da.empty((dims[2,2],dims[2,3]))).compute(),
             'PAR':dd.from_array(da.empty((dims[3,2],dims[3,3]))).compute(),
             'Kd':dd.from_array(da.empty((dims[4,2],dims[4,3]))).compute(),
             'POC':dd.from_array(da.empty((dims[5,2],dims[5,3]))).compute(),
             'FLH':dd.from_array(da.empty((dims[6,2],dims[6,3]))).compute(),
             'iPAR':dd.from_array(da.empty((dims[7,2],dims[7,3]))).compute(),
             'CDOM':dd.from_array(da.empty((dims[8,2],dims[8,3]))).compute(),
             'SSHA':dd.from_array(da.empty((dims[9,2],dims[9,3]))).compute(),
             'NPP':dd.from_array(da.empty((dims[10,2],dims[10,3]))).compute(),
             'MLD':dd.from_array(da.empty((dims[11,2],dims[11,3]))).compute(),
             'SAL':dd.from_array(da.empty((dims[12,2],dims[12,3]))).compute()}

In [ ]:
#-----------------------------------------------------------------------------
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print('Beginning extraction loop...')
start = timeit.default_timer() # start the clock
for count, (k, folder) in enumerate(zip(variables, folders)):
    print('\nAccessing ' + '"' + folder + '"' + ' folder...')
    print()
    path = os.path.join(directory, folder)
    if count>=11: # My PC is RAM limited so this line bypasses looping at higher res (0.25x0.25) below to extract one variable at a time, saving memory space; if computing resources are not limited, comment out to extract all variables at once
      # if count == 0 and k==0: # My PC is RAM limited so this line bypasses looping at higher res (0.25x0.25) below to extract one variable at a time, saving memory space; if computing resources are not limited, comment out to extract all variables at once
      for subdir, dirs, files in os.walk(path):
          for n, file in enumerate(files):
              filepath = subdir + os.sep + file
              # Print status: starting iteration
              print(folder + ' (' + str(count+1) + ' of ' + str(np.size(folders)) + ' folders)' + ': Extracting... ' + str(n+1) +' of ' + str(np.size(files)))
              #-----------------------------------------------------------------
              # Extract data
              if count < 11:
                  if count < 9:
                      # Open netCDF file
                      vars_ = xr.open_dataset(filepath)
                      if count == 0:
                          # Chl:
                          data = vars_.chlor_a.values
                      if count == 1:
                          # PIC:
                          data = vars_.pic.values
                      if count == 2:
                          # SST:
                          data = vars_.sst.values
                      if count == 3:
                          # PAR:
                          data = vars_.par.values
                      if count == 4:
                          # Kd (490 nm)
                          data = vars_.Kd_490.values
                      if count == 5:
                          # POC
                          data = vars_.poc.values
                      if count == 6:
                          # FLH
                          data = vars_.nflh.values
                      if count == 7:
                          # iPAR
                          data = vars_.ipar.values
                      if count == 8:
                          # CDOM
                          data = vars_.adg_443_giop.values
                      lat = vars_.coords['lat'].values
                      lon = vars_.coords['lon'].values
                      # time = pd.to_datetime(vars_.time_coverage_start)+datetime.timedelta(days=2) # this adds 2 days to correct for SeaWiFS files starting on the last day of previous month
                      time = int((pd.to_datetime(vars_.time_coverage_start)+datetime.timedelta(days=2)).strftime('%m')) # this adds 2 days to correct for SeaWiFS files starting on the last day of previous month
                  if count == 9:
                      # SSHA
                      # Open netCDF file
                      vars_ = xr.open_dataset(filepath)
                      data = vars_.SLA.values[0,:,:].T # transpose to arrange as lat x lon
                      lat = vars_.coords['Latitude'].values
                      lon = vars_.coords['Longitude'].values
                      lon = pd.Series(lon).where(lon<180, lon-360).values # convert to W/E coords
                      # time = pd.to_datetime(vars_.coords['Time'].values)
                      time = int(pd.to_datetime(vars_.coords['Time'].values).strftime('%m').values)
                  if count == 10:
                      # NPP
                      vars_ = SD(filepath, SDC.READ)
                      data = np.flipud(vars_.select('npp').get())
                      ndeg = 180/1080
                      lat = np.arange(-90,90,ndeg)
                      lon = np.arange(-180,180,ndeg)
                      # extract date from julian day in filename:
                      julian_date = filepath.split(".")[1]
                      # time = pd.to_datetime(datetime.datetime.strptime(julian_date[2:], '%y%j').date()) #strftime requires year to be cropped to last 2 digits
                      time = int(pd.to_datetime(datetime.datetime.strptime(julian_date[2:], '%y%j').date()).strftime('%m')) #strftime requires year to be cropped to last 2 digits
                  #-----------------------------------------------------------------
                  # case 1: match pixels to high resolution satellite data
                  if lon[1]-lon[0] >= grid: #i.e. if upsampling to finer grid size
                      # Regrid data and interpolate though:
                      # First find the indices to index by lat/lon
                      latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                      loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)

                      # Restrict the data to the specified lat/lons
                      data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]

                      # Create data matrix with coordinates
                      data_mat = pd.DataFrame(data_indexed)
                      data_mat.columns = lon[loninds].flatten()
                      data_mat.index = lat[latinds].flatten()
                      #sort the data so that columns are ascending (matches 'new_shape' below)
                      data_mat = data_mat.reindex(sorted(data_mat.columns), axis=1) # sort columns in ascending order

                      # Create a new matrix with new gridded coordinates
                      lat_new = np.arange(min_lat, max_lat+grid, grid)
                      lon_new = np.arange(min_lon, max_lon+grid, grid)
                      new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
                      new_shape.columns=lon_new
                      new_shape.index=lat_new

                      # Now reindex data to new coordinates - add in NaNs to interpolate through
                      # important: find corresponding nearest value in new index/columns to replace data coords by
                      new_idx = [new_shape.index[abs(new_shape.index.values-i).argmin()] for i in data_mat.index]
                      new_cols = [new_shape.columns[abs(new_shape.columns.values-i).argmin()] for i in data_mat.columns]
                      # now rename idx/cols with new values in the data matrix (this allows the reindex_like function to properly map and insert nans below)
                      data_mat.set_axis(new_idx, axis=0, inplace=True)
                      data_mat.set_axis(new_cols, axis=1, inplace=True)

                      data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
                      data_proc = data_proc.rename(columns={0:'data'})
                      data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
                      data_proc['datetime'] = np.tile(time,data_proc.shape[0])
                      data_proc.reset_index(inplace=True)
                      data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
                      # data_proc.reset_index(inplace=True)
                      
                  #-----------------------------------------------------------------
                  # case 2: downsample to courser grid
                  if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
                      # Bin the data
                      
                      # First find the indices to index by lat/lon
                      latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                      loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                      
                      # Restrict the data to the NE Pacific lat/lons
                      data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]
                      
                      # Convert time/lats/lons into repeating matrices to match data dimensions
                      lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                      lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                      time_mat = np.tile(time, len(np.ravel(lat_mat)))
                      
                      # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                      # Need these as pandas dataframes to using binning scheme:
                      d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
                      data_long = pd.DataFrame(data=d)
                      
                      # Bin data as averages across gridded spatial bins:
                      to_bin = lambda x: np.round(x / grid) * grid
                      data_long['latbins'] = data_long.lat.map(to_bin)
                      data_long['lonbins'] = data_long.lon.map(to_bin)
                      data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                      
                      # Rename binned columns + drop mean lat/lons:
                      data_proc = data_proc.drop(columns=['lat', 'lon'])
                      # data_proc.reset_index(inplace=True) # remove index specification on columns
              #-----------------------------------------------------------------
              # Lets extract MIMOC data...
              if count >= 11:
                  if count == 11:
                      # Open netCDF file
                      vars_ = xr.open_dataset(filepath)
                      data = vars_.DEPTH_MIXED_LAYER.values
                  if count == 12:
                      # Open netCDF file
                      vars_ = xr.open_dataset(filepath)
                      data = vars_.SALINITY[0,:,:].values # select values at surface layer (0 m)
                  lat = vars_.LATITUDE.values
                  lon = vars_.LONGITUDE.values
                  lon = pd.Series(lon).where(lon<180, lon-360).values # convert to W/E coords
                  julian_date = file.split("month")[1].split('.nc')[0]
                  time = int(julian_date) #strftime requires year to be cropped to last 2 digits
                  #-----------------------------------------------------------------
                  # case 1: match pixels to high resolution satellite data
                  if lon[1]-lon[0] >= grid: #i.e. if upsampling to finer grid size
                      # Regrid data and interpolate though:
                      # First find the indices to index by lat/lon
                      latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                      loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)

                      # Restrict the data to the specified lat/lons
                      data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]

                      # Create data matrix with coordinates
                      data_mat = pd.DataFrame(data_indexed)
                      data_mat.columns = lon[loninds].flatten()
                      data_mat.index = lat[latinds].flatten()
                      #sort the data so that columns are ascending (matches 'new_shape' below)
                      data_mat = data_mat.reindex(sorted(data_mat.columns), axis=1) # sort columns in ascending order

                      # Create a new matrix with new gridded coordinates
                      lat_new = np.arange(min_lat, max_lat+grid, grid)
                      lon_new = np.arange(min_lon, max_lon+grid, grid)
                      new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
                      new_shape.columns=lon_new
                      new_shape.index=lat_new

                      # Now reindex data to new coordinates - add in NaNs to interpolate through
                      # important: find corresponding nearest value in new index/columns to replace data coords by
                      new_idx = [new_shape.index[abs(new_shape.index.values-i).argmin()] for i in data_mat.index]
                      new_cols = [new_shape.columns[abs(new_shape.columns.values-i).argmin()] for i in data_mat.columns]
                      # now rename idx/cols with new values in the data matrix (this allows the reindex_like function to properly map and insert nans below)
                      data_mat.set_axis(new_idx, axis=0, inplace=True)
                      data_mat.set_axis(new_cols, axis=1, inplace=True)

                      data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
                      data_proc = data_proc.rename(columns={0:'data'})
                      data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
                      data_proc['datetime'] = np.tile(time,data_proc.shape[0])
                      data_proc.reset_index(inplace=True)
                      data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
                      # data_proc.reset_index(inplace=True)
                      
                  #-----------------------------------------------------------------
                  # case 2: downsample to courser grid
                  if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
                      # Bin the data
                      
                      # First find the indices to index by lat/lon
                      latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                      loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                      
                      # Restrict the data to the NE Pacific lat/lons
                      data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]
                      
                      # Convert time/lats/lons into repeating matrices to match data dimensions
                      lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                      lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                      time_mat = np.tile(time, len(np.ravel(lat_mat)))
                      
                      # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                      # Need these as pandas dataframes to using binning scheme:
                      d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
                      data_long = pd.DataFrame(data=d)
                      
                      # Bin data as averages across gridded spatial bins:
                      to_bin = lambda x: np.round(x / grid) * grid
                      data_long['latbins'] = data_long.lat.map(to_bin)
                      data_long['lonbins'] = data_long.lon.map(to_bin)
                      data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                      
                      # Rename binned columns + drop mean lat/lons:
                      data_proc = data_proc.drop(columns=['lat', 'lon'])
                      # data_proc.reset_index(inplace=True) # remove index specification on columns
              if count <= 9:
                # close xarray netCDF file connection 
                vars_.close()            
              # Save to output array
              if n == 0: # for first iteration, define pre-allocated variable in dict as new binned data
                  variables[k] = data_proc
              else: # for subsequent iterations, add to existing dict variable by "stacking" new binned datasets row-wise
                  # data_proc = data_proc.set_index(['datetime','latbins','lonbins'])
                  variables[k] = pd.concat([variables[k],data_proc]).groupby(['datetime','latbins','lonbins']).mean()
              
              # Print status: completed iteration
              print(folder + ' (' + str(count+1) + ' of ' + str(np.size(folders)) + ' folders)' + ': Completed ' + str(n+1) + ' of ' + str(np.size(files)))
      print('\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
      print('Ta-da! ' + str(count+1) + ' of ' + str(np.size(folders)) + ' datasets processed')
end = timeit.default_timer() # stop the clock
print('\n~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~')
print('Execution time:') # output computation times:
print(str(round(end-start,5)),'secs')
print(str(round((end-start)/60,5)),'mins')
print(str(round((end-start)/3600,5)),'hrs')
del vars_, data_long, data, data_indexed, data_proc, time_mat

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Beginning extraction loop...

Accessing "1_Chl" folder...


Accessing "2_PIC" folder...


Accessing "3_SST" folder...


Accessing "4_PAR" folder...


Accessing "5_Kd" folder...


Accessing "6_POC" folder...


Accessing "7_nFLH" folder...


Accessing "8_iPAR" folder...


Accessing "91_CDOM" folder...


Accessing "92_SSHA" folder...


Accessing "93_NPP" folder...


Accessing "94_MIMOC_MLD" folder...

94_MIMOC_MLD (12 of 13 folders): Extracting... 1 of 7
94_MIMOC_MLD (12 of 13 folders): Completed 1 of 7
94_MIMOC_MLD (12 of 13 folders): Extracting... 2 of 7
94_MIMOC_MLD (12 of 13 folders): Completed 2 of 7
94_MIMOC_MLD (12 of 13 folders): Extracting... 3 of 7
94_MIMOC_MLD (12 of 13 folders): Completed 3 of 7
94_MIMOC_MLD (12 of 13 folders): Extracting... 4 of 7
94_MIMOC_MLD (12 of 13 folders): Completed 4 of 7
94_MIMOC_MLD (12 of 13 folders): Extracting... 5 of 7
94_MIMOC_MLD (12 of 13 folders): Completed 5 of 7
94_MIMOC_MLD (12 of 13 folders): Extracting..

NameError: ignored

In [ ]:
# Download processed files
from google.colab import files

for i, var_ in enumerate(variables):
  # if var_ == 'POC':
  if i >= 11:
    variables[var_].to_csv(write_to+str(var_)+'_'+str(round(grid,3))+'_deg.csv')
    files.download(write_to+str(var_)+'_'+str(round(grid,3))+'_deg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#%% Extracting COPERNICUS wind field data (singular netCDF; 0.25x0.25)

print('\nExtracting Copernicus wind field data...')
print('\nLoading data...')
# Pre-allocate dask array to store new binned data to
wind = dd.from_array(da.empty((1,6)))
# Load up wind data file (Copernicus timeseries data is downloaded into a single netCDF)
windfile = xr.open_dataset('CERSAT-GLO-REP_WIND_L4-OBS_FULL_TIME_SERIE_1607246388558.nc')

# Define variables
# shape is (time,depth,lat,lon)
lat = windfile.coords['latitude'].values
lon = windfile.coords['longitude'].values
Us = windfile.eastward_wind.values[:,0,:,:]
Vs = windfile.northward_wind.values[:,0,:,:]
wspd = windfile.wind_speed.values[:,0,:,:]
timerange = pd.to_datetime(windfile.time.values,utc=True).strftime('%Y%m').to_series().reset_index(drop=True)


# Define indices
latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
if which_dataset == 'DMS':
  timeinds = ['200112', '200201', '200202', '200301', '200302', '200303',
              '200310', '200311', '200401', '200402', '200412', '200501',
              '200502', '200510', '200511', '200512', '200601', '200602',
              '200611', '200612', '200711', '200712', '200801', '200802',
              '200803', '200804', '200812', '200901', '200902', '201012',
              '201202', '201203', '201311', '201312', '201402', '201403'][20:] # index from 2007 onwards to match data
else: # if Fe
  timeinds = ['200707', '200708', '200709', '200711', '200712', '200802',
              '200803', '200804', '200912', '201005', '201006', '201007',
              '201010', '201011', '201103', '201105', '201208', '201209',
              '201210']

idx = np.arange(0,timerange.size,1)
times = pd.DataFrame(idx,index=timerange)
timepts = times.loc[timeinds].values


print('\nBegin extraction loop...')
# Loop to bin/reshape
for n,pt in enumerate(timepts[:,0]):
    print(str(n+1)+' of '+str(np.size(timepts[:,0]))+' iterations...')
    #===========================================================================
    #                             Wind speed
    #===========================================================================
    # case 1: match pixels to high resolution satellite data
    if lon[1]-lon[0] >= grid: #i.e. if upsampling to finer grid size
        # Regrid data and interpolate though:
        # First find the indices to index by lat/lon
        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)

        # Restrict the data to the specified lat/lons
        data_indexed = wspd[pt,:,:][latinds[:,None], loninds[None,:]][:,:,0]

        # Create data matrix with coordinates
        data_mat = pd.DataFrame(data_indexed)
        data_mat.columns = lon[loninds].flatten()
        data_mat.index = lat[latinds].flatten()
        #sort the data so that columns are ascending (matches 'new_shape' below)
        data_mat = data_mat.reindex(sorted(data_mat.columns), axis=1) # sort columns in ascending order

        # Create a new matrix with new gridded coordinates
        lat_new = np.arange(min_lat, max_lat+grid, grid)
        lon_new = np.arange(min_lon, max_lon+grid, grid)
        new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
        new_shape.columns=lon_new
        new_shape.index=lat_new

        # Now reindex data to new coordinates - add in NaNs to interpolate through
        # important: find corresponding nearest value in new index/columns to replace data coords by
        new_idx = [new_shape.index[abs(new_shape.index.values-i).argmin()] for i in data_mat.index]
        new_cols = [new_shape.columns[abs(new_shape.columns.values-i).argmin()] for i in data_mat.columns]
        # now rename idx/cols with new values in the data matrix (this allows the reindex_like function to properly map and insert nans below)
        data_mat.set_axis(new_idx, axis=0, inplace=True)
        data_mat.set_axis(new_cols, axis=1, inplace=True)

        data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
        data_proc = data_proc.rename(columns={0:'data'})
        data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
        data_proc['datetime'] = np.tile(int(pd.to_datetime(datetime.datetime.strptime(timeinds[n], '%Y%m').strftime('%Y-%m')).strftime('%m')),data_proc.shape[0])
        data_proc.reset_index(inplace=True)
        data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
        # data_proc.reset_index(inplace=True)
        
    #-----------------------------------------------------------------
    # case 2: downsample to courser grid
    if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
        # Bin the data
        
        # First find the indices to index by lat/lon
        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
        
        # Restrict the data to the NE Pacific lat/lons
        data_indexed = wspd[pt,:,:][latinds[:,None], loninds[None,:]][:,:,0]
        
        # Convert time/lats/lons into repeating matrices to match data dimensions
        lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
        lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
        time_mat = np.tile(int(pd.to_datetime(datetime.datetime.strptime(timeinds[n], '%Y%m').strftime('%Y-%m')).strftime('%m')), len(np.ravel(lat_mat)))
        
        # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
        # Need these as pandas dataframes to using binning scheme:
        d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
        data_long = pd.DataFrame(data=d)
        
        # Bin data as averages across gridded spatial bins:
        to_bin = lambda x: np.round(x / grid) * grid
        data_long['latbins'] = data_long.lat.map(to_bin)
        data_long['lonbins'] = data_long.lon.map(to_bin)
        data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
        
        # Rename binned columns + drop mean lat/lons:
        data_proc = data_proc.drop(columns=['lat', 'lon'])
    # Save to output array
    if n == 0: # for first iteration, define pre-allocated variable in dict as new binned data
        wind = data_proc
    else: # for subsequent iterations, add to existing dict variable by "stacking" new binned datasets row-wise
        wind = pd.concat([wind,data_proc]).groupby(['datetime','latbins','lonbins']).mean()
    #===========================================================================
    #                             U component
    #===========================================================================
    # case 1: match pixels to high resolution satellite data
    if lon[1]-lon[0] >= grid: #i.e. if upsampling to finer grid size
        # Regrid data and interpolate though:
        # First find the indices to index by lat/lon
        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)

        # Restrict the data to the specified lat/lons
        data_indexed = Us[pt,:,:][latinds[:,None], loninds[None,:]][:,:,0]

        # Create data matrix with coordinates
        data_mat = pd.DataFrame(data_indexed)
        data_mat.columns = lon[loninds].flatten()
        data_mat.index = lat[latinds].flatten()
        #sort the data so that columns are ascending (matches 'new_shape' below)
        data_mat = data_mat.reindex(sorted(data_mat.columns), axis=1) # sort columns in ascending order

        # Create a new matrix with new gridded coordinates
        lat_new = np.arange(min_lat, max_lat+grid, grid)
        lon_new = np.arange(min_lon, max_lon+grid, grid)
        new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
        new_shape.columns=lon_new
        new_shape.index=lat_new

        # Now reindex data to new coordinates - add in NaNs to interpolate through
        # important: find corresponding nearest value in new index/columns to replace data coords by
        new_idx = [new_shape.index[abs(new_shape.index.values-i).argmin()] for i in data_mat.index]
        new_cols = [new_shape.columns[abs(new_shape.columns.values-i).argmin()] for i in data_mat.columns]
        # now rename idx/cols with new values in the data matrix (this allows the reindex_like function to properly map and insert nans below)
        data_mat.set_axis(new_idx, axis=0, inplace=True)
        data_mat.set_axis(new_cols, axis=1, inplace=True)

        data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
        data_proc = data_proc.rename(columns={0:'data'})
        data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
        data_proc['datetime'] = np.tile(int(pd.to_datetime(datetime.datetime.strptime(timeinds[n], '%Y%m').strftime('%Y-%m')).strftime('%m')),data_proc.shape[0])
        data_proc.reset_index(inplace=True)
        data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
        # data_proc.reset_index(inplace=True)
        
    #-----------------------------------------------------------------
    # case 2: downsample to courser grid
    if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
        # Bin the data
        
        # First find the indices to index by lat/lon
        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
        
        # Restrict the data to the NE Pacific lat/lons
        data_indexed = Us[pt,:,:][latinds[:,None], loninds[None,:]][:,:,0]
        
        # Convert time/lats/lons into repeating matrices to match data dimensions
        lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
        lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
        time_mat = np.tile(int(pd.to_datetime(datetime.datetime.strptime(timeinds[n], '%Y%m').strftime('%Y-%m')).strftime('%m')), len(np.ravel(lat_mat)))
        
        # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
        # Need these as pandas dataframes to using binning scheme:
        d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
        data_long = pd.DataFrame(data=d)
        
        # Bin data as averages across gridded spatial bins:
        to_bin = lambda x: np.round(x / grid) * grid
        data_long['latbins'] = data_long.lat.map(to_bin)
        data_long['lonbins'] = data_long.lon.map(to_bin)
        data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
        
        # Rename binned columns + drop mean lat/lons:
        data_proc = data_proc.drop(columns=['lat', 'lon'])
    # Save to output array
    if n == 0: # for first iteration, define pre-allocated variable in dict as new binned data
        U = data_proc
    else: # for subsequent iterations, add to existing dict variable by "stacking" new binned datasets row-wise
        U = pd.concat([U,data_proc]).groupby(['datetime','latbins','lonbins']).mean()
    #===========================================================================
    #                             V component
    #===========================================================================
    # case 1: match pixels to high resolution satellite data
    if lon[1]-lon[0] >= grid: #i.e. if upsampling to finer grid size
        # Regrid data and interpolate though:
        # First find the indices to index by lat/lon
        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)

        # Restrict the data to the specified lat/lons
        data_indexed = Vs[pt,:,:][latinds[:,None], loninds[None,:]][:,:,0]

        # Create data matrix with coordinates
        data_mat = pd.DataFrame(data_indexed)
        data_mat.columns = lon[loninds].flatten()
        data_mat.index = lat[latinds].flatten()
        #sort the data so that columns are ascending (matches 'new_shape' below)
        data_mat = data_mat.reindex(sorted(data_mat.columns), axis=1) # sort columns in ascending order

        # Create a new matrix with new gridded coordinates
        lat_new = np.arange(min_lat, max_lat+grid, grid)
        lon_new = np.arange(min_lon, max_lon+grid, grid)
        new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
        new_shape.columns=lon_new
        new_shape.index=lat_new

        # Now reindex data to new coordinates - add in NaNs to interpolate through
        # important: find corresponding nearest value in new index/columns to replace data coords by
        new_idx = [new_shape.index[abs(new_shape.index.values-i).argmin()] for i in data_mat.index]
        new_cols = [new_shape.columns[abs(new_shape.columns.values-i).argmin()] for i in data_mat.columns]
        # now rename idx/cols with new values in the data matrix (this allows the reindex_like function to properly map and insert nans below)
        data_mat.set_axis(new_idx, axis=0, inplace=True)
        data_mat.set_axis(new_cols, axis=1, inplace=True)

        data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
        data_proc = data_proc.rename(columns={0:'data'})
        data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
        data_proc['datetime'] = np.tile(int(pd.to_datetime(datetime.datetime.strptime(timeinds[n], '%Y%m').strftime('%Y-%m')).strftime('%m')),data_proc.shape[0])
        data_proc.reset_index(inplace=True)
        data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
        # data_proc.reset_index(inplace=True)
        
    #-----------------------------------------------------------------
    # case 2: downsample to courser grid
    if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
        # Bin the data
        
        # First find the indices to index by lat/lon
        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
        
        # Restrict the data to the NE Pacific lat/lons
        data_indexed = Vs[pt,:,:][latinds[:,None], loninds[None,:]][:,:,0]
        
        # Convert time/lats/lons into repeating matrices to match data dimensions
        lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
        lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
        time_mat = np.tile(int(pd.to_datetime(datetime.datetime.strptime(timeinds[n], '%Y%m').strftime('%Y-%m')).strftime('%m')), len(np.ravel(lat_mat)))
        
        # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
        # Need these as pandas dataframes to using binning scheme:
        d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
        data_long = pd.DataFrame(data=d)
        
        # Bin data as averages across gridded spatial bins:
        to_bin = lambda x: np.round(x / grid) * grid
        data_long['latbins'] = data_long.lat.map(to_bin)
        data_long['lonbins'] = data_long.lon.map(to_bin)
        data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
        
        # Rename binned columns + drop mean lat/lons:
        data_proc = data_proc.drop(columns=['lat', 'lon'])
    # Save to output array
    if n == 0: # for first iteration, define pre-allocated variable in dict as new binned data
        V = data_proc
    else: # for subsequent iterations, add to existing dict variable by "stacking" new binned datasets row-wise
        V = pd.concat([V,data_proc]).groupby(['datetime','latbins','lonbins']).mean()
    print(str(n+1)+' of '+str(np.size(timepts[:,0]))+' iterations completed!')
del windfile
#===============================================================================
# write data to file
from google.colab import files
wind.to_csv(write_to+'wind_copernicus'+'_'+str(round(grid,3))+'_deg.csv')
U.to_csv(write_to+'U_copernicus'+'_'+str(round(grid,3))+'_deg.csv')
V.to_csv(write_to+'V_copernicus'+'_'+str(round(grid,3))+'_deg.csv')
files.download(write_to+'wind_copernicus'+'_'+str(round(grid,3))+'_deg.csv')
files.download(write_to+'U_copernicus'+'_'+str(round(grid,3))+'_deg.csv')
files.download(write_to+'V_copernicus'+'_'+str(round(grid,3))+'_deg.csv')


Extracting Copernicus wind field data...

Loading data...

Begin extraction loop...
1 of 16 iterations...
1 of 16 iterations completed!
2 of 16 iterations...
2 of 16 iterations completed!
3 of 16 iterations...
3 of 16 iterations completed!
4 of 16 iterations...
4 of 16 iterations completed!
5 of 16 iterations...
5 of 16 iterations completed!
6 of 16 iterations...
6 of 16 iterations completed!
7 of 16 iterations...
7 of 16 iterations completed!
8 of 16 iterations...
8 of 16 iterations completed!
9 of 16 iterations...
9 of 16 iterations completed!
10 of 16 iterations...
10 of 16 iterations completed!
11 of 16 iterations...
11 of 16 iterations completed!
12 of 16 iterations...
12 of 16 iterations completed!
13 of 16 iterations...
13 of 16 iterations completed!
14 of 16 iterations...
14 of 16 iterations completed!
15 of 16 iterations...
15 of 16 iterations completed!
16 of 16 iterations...
16 of 16 iterations completed!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#%% Process CCMP wind speed data (multiple netcdfs, 0.25x0.25o)
if which_dataset == 'DMS':
  
  wind_dir = 'CCMP_0.25_wind_speeds' # E: or C:, depending if using hard drive/thumb drive

  files = sorted(os.listdir(wind_dir)) # calls specific datasets grouped in folders
  # path = os.path.join(wind_dir, folder)

  for subdir, dirs, files in os.walk(wind_dir):
          for n, file in enumerate(files):
            filepath = subdir + os.sep + file
            #-------------------------------------------------------------------
            # wind speed
            vars_ = xr.open_dataset(filepath)
            data = vars_.wspd.values[0,:,:]
            lat = vars_.lat.values
            lon = vars_.lon.values
            lon = pd.Series(lon).where(lon<180, lon-360).values # convert to W/E coords
            time = int(pd.to_datetime(vars_.time.values).strftime('%m').values)
            if lon[1]-lon[0] >= grid: #i.e. if upsampling to finer grid size
                # Regrid data and interpolate though:
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)

                # Restrict the data to the specified lat/lons
                data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]

                # Create data matrix with coordinates
                data_mat = pd.DataFrame(data_indexed)
                data_mat.columns = lon[loninds].flatten()
                data_mat.index = lat[latinds].flatten()
                #sort the data so that columns are ascending (matches 'new_shape' below)
                data_mat = data_mat.reindex(sorted(data_mat.columns), axis=1) # sort columns in ascending order

                # Create a new matrix with new gridded coordinates
                lat_new = np.arange(min_lat, max_lat+grid, grid)
                lon_new = np.arange(min_lon, max_lon+grid, grid)
                new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
                new_shape.columns=lon_new
                new_shape.index=lat_new

                # Now reindex data to new coordinates - add in NaNs to interpolate through
                # important: find corresponding nearest value in new index/columns to replace data coords by
                new_idx = [new_shape.index[abs(new_shape.index.values-i).argmin()] for i in data_mat.index]
                new_cols = [new_shape.columns[abs(new_shape.columns.values-i).argmin()] for i in data_mat.columns]
                # now rename idx/cols with new values in the data matrix (this allows the reindex_like function to properly map and insert nans below)
                data_mat.set_axis(new_idx, axis=0, inplace=True)
                data_mat.set_axis(new_cols, axis=1, inplace=True)

                data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
                data_proc = data_proc.rename(columns={0:'data'})
                data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
                data_proc['datetime'] = np.tile(time,data_proc.shape[0])
                data_proc.reset_index(inplace=True)
                data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
                # data_proc.reset_index(inplace=True)
                
            #-----------------------------------------------------------------
            # case 2: downsample to courser grid
            if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
                # Bin the data
                
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                
                # Restrict the data to the NE Pacific lat/lons
                data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]
                
                # Convert time/lats/lons into repeating matrices to match data dimensions
                lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                time_mat = np.tile(time, len(np.ravel(lat_mat)))
                
                # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                # Need these as pandas dataframes to using binning scheme:
                d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
                data_long = pd.DataFrame(data=d)
                
                # Bin data as averages across gridded spatial bins:
                to_bin = lambda x: np.round(x / grid) * grid
                data_long['latbins'] = data_long.lat.map(to_bin)
                data_long['lonbins'] = data_long.lon.map(to_bin)
                data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                
                # Rename binned columns + drop mean lat/lons:
                data_proc = data_proc.drop(columns=['lat', 'lon'])
            # Save to output array
            if n == 0: # for first iteration, define pre-allocated variable in dict as new binned data
              wind = data_proc
            else: # for subsequent iterations, add to existing dict variable by "stacking" new binned datasets row-wise
              wind = pd.concat([wind,data_proc]).groupby(['datetime','latbins','lonbins']).mean()
            #-------------------------------------------------------------------
            # U vector
            vars_ = xr.open_dataset(filepath)
            data = vars_.uwnd.values[0,:,:]
            lat = vars_.lat.values
            lon = vars_.lon.values
            lon = pd.Series(lon).where(lon<180, lon-360).values # convert to W/E coords
            time = int(pd.to_datetime(vars_.time.values).strftime('%m').values)
            if lon[1]-lon[0] >= grid: #i.e. if upsampling to finer grid size
                # Regrid data and interpolate though:
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)

                # Restrict the data to the specified lat/lons
                data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]

                # Create data matrix with coordinates
                data_mat = pd.DataFrame(data_indexed)
                data_mat.columns = lon[loninds].flatten()
                data_mat.index = lat[latinds].flatten()
                #sort the data so that columns are ascending (matches 'new_shape' below)
                data_mat = data_mat.reindex(sorted(data_mat.columns), axis=1) # sort columns in ascending order

                # Create a new matrix with new gridded coordinates
                lat_new = np.arange(min_lat, max_lat+grid, grid)
                lon_new = np.arange(min_lon, max_lon+grid, grid)
                new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
                new_shape.columns=lon_new
                new_shape.index=lat_new

                # Now reindex data to new coordinates - add in NaNs to interpolate through
                # important: find corresponding nearest value in new index/columns to replace data coords by
                new_idx = [new_shape.index[abs(new_shape.index.values-i).argmin()] for i in data_mat.index]
                new_cols = [new_shape.columns[abs(new_shape.columns.values-i).argmin()] for i in data_mat.columns]
                # now rename idx/cols with new values in the data matrix (this allows the reindex_like function to properly map and insert nans below)
                data_mat.set_axis(new_idx, axis=0, inplace=True)
                data_mat.set_axis(new_cols, axis=1, inplace=True)

                data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
                data_proc = data_proc.rename(columns={0:'data'})
                data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
                data_proc['datetime'] = np.tile(time,data_proc.shape[0])
                data_proc.reset_index(inplace=True)
                data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
                # data_proc.reset_index(inplace=True)
                
            #-----------------------------------------------------------------
            # case 2: downsample to courser grid
            if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
                # Bin the data
                
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                
                # Restrict the data to the NE Pacific lat/lons
                data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]
                
                # Convert time/lats/lons into repeating matrices to match data dimensions
                lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                time_mat = np.tile(time, len(np.ravel(lat_mat)))
                
                # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                # Need these as pandas dataframes to using binning scheme:
                d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
                data_long = pd.DataFrame(data=d)
                
                # Bin data as averages across gridded spatial bins:
                to_bin = lambda x: np.round(x / grid) * grid
                data_long['latbins'] = data_long.lat.map(to_bin)
                data_long['lonbins'] = data_long.lon.map(to_bin)
                data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                
                # Rename binned columns + drop mean lat/lons:
                data_proc = data_proc.drop(columns=['lat', 'lon'])
            # Save to output array
            if n == 0: # for first iteration, define pre-allocated variable in dict as new binned data
              U = data_proc
            else: # for subsequent iterations, add to existing dict variable by "stacking" new binned datasets row-wise
              U = pd.concat([U,data_proc]).groupby(['datetime','latbins','lonbins']).mean()
            #-------------------------------------------------------------------
            # V vector
            vars_ = xr.open_dataset(filepath)
            data = vars_.vwnd.values[0,:,:]
            lat = vars_.lat.values
            lon = vars_.lon.values
            lon = pd.Series(lon).where(lon<180, lon-360).values # convert to W/E coords
            time = int(pd.to_datetime(vars_.time.values).strftime('%m').values)
            if lon[1]-lon[0] >= grid: #i.e. if upsampling to finer grid size
                # Regrid data and interpolate though:
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)

                # Restrict the data to the specified lat/lons
                data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]

                # Create data matrix with coordinates
                data_mat = pd.DataFrame(data_indexed)
                data_mat.columns = lon[loninds].flatten()
                data_mat.index = lat[latinds].flatten()
                #sort the data so that columns are ascending (matches 'new_shape' below)
                data_mat = data_mat.reindex(sorted(data_mat.columns), axis=1) # sort columns in ascending order

                # Create a new matrix with new gridded coordinates
                lat_new = np.arange(min_lat, max_lat+grid, grid)
                lon_new = np.arange(min_lon, max_lon+grid, grid)
                new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
                new_shape.columns=lon_new
                new_shape.index=lat_new

                # Now reindex data to new coordinates - add in NaNs to interpolate through
                # important: find corresponding nearest value in new index/columns to replace data coords by
                new_idx = [new_shape.index[abs(new_shape.index.values-i).argmin()] for i in data_mat.index]
                new_cols = [new_shape.columns[abs(new_shape.columns.values-i).argmin()] for i in data_mat.columns]
                # now rename idx/cols with new values in the data matrix (this allows the reindex_like function to properly map and insert nans below)
                data_mat.set_axis(new_idx, axis=0, inplace=True)
                data_mat.set_axis(new_cols, axis=1, inplace=True)

                data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
                data_proc = data_proc.rename(columns={0:'data'})
                data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
                data_proc['datetime'] = np.tile(time,data_proc.shape[0])
                data_proc.reset_index(inplace=True)
                data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
                # data_proc.reset_index(inplace=True)
                
            #-----------------------------------------------------------------
            # case 2: downsample to courser grid
            if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
                # Bin the data
                
                # First find the indices to index by lat/lon
                latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
                loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
                
                # Restrict the data to the NE Pacific lat/lons
                data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]
                
                # Convert time/lats/lons into repeating matrices to match data dimensions
                lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
                lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
                time_mat = np.tile(time, len(np.ravel(lat_mat)))
                
                # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
                # Need these as pandas dataframes to using binning scheme:
                d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
                data_long = pd.DataFrame(data=d)
                
                # Bin data as averages across gridded spatial bins:
                to_bin = lambda x: np.round(x / grid) * grid
                data_long['latbins'] = data_long.lat.map(to_bin)
                data_long['lonbins'] = data_long.lon.map(to_bin)
                data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
                
                # Rename binned columns + drop mean lat/lons:
                data_proc = data_proc.drop(columns=['lat', 'lon'])
            # Save to output array
            if n == 0: # for first iteration, define pre-allocated variable in dict as new binned data
              V = data_proc
            else: # for subsequent iterations, add to existing dict variable by "stacking" new binned datasets row-wise
              V = pd.concat([V,data_proc]).groupby(['datetime','latbins','lonbins']).mean()
              
  #=============================================================================
  # write data to file
  from google.colab import files
  wind.to_csv(write_to+'wind_CCMP'+'_'+str(round(grid,3))+'_deg.csv')
  U.to_csv(write_to+'U_CCMP'+'_'+str(round(grid,3))+'_deg.csv')
  V.to_csv(write_to+'V_CCMP'+'_'+str(round(grid,3))+'_deg.csv')
  files.download(write_to+'wind_CCMP'+'_'+str(round(grid,3))+'_deg.csv')
  files.download(write_to+'U_CCMP'+'_'+str(round(grid,3))+'_deg.csv')
  files.download(write_to+'V_CCMP'+'_'+str(round(grid,3))+'_deg.csv')
else: # Fe data doesn't date back past 2007, so ignore this cell
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#%% Processing WOA18 data (csv; 1x1 to 0.25x0.25)

print('\nProcessing WOA18 SSN data...')
print('\nLoading data...')
# if DMS
if which_dataset == 'DMS':
  months = [1,2,3,4,10,11,12]
else: # if Fe
  months = [2,3,4,5,6,7,8,9,10,11,12]
#------------------------------------------------------------------------------
#### Nitrate
#------------------------------------------------------------------------------
# Pre-allocate dask array to store new binned data to
SSN = dd.from_array(da.empty((1,len(months))))

for n,pt in enumerate(months):
    # Load up wind data file (Copernicus timeseries data is downloaded into a single netCDF)
    if pt<10:
      data = pd.read_csv('WOA18/woa18_all_n0'+str(pt)+'an01.csv', header=[1]).reset_index().drop('index', axis=1).rename(columns={'#COMMA SEPARATED LATITUDE':'lat',' LONGITUDE':'lon', ' AND VALUES AT DEPTHS (M):0':0}).set_index(['lat','lon']).loc[:,0]
    else:
      data = pd.read_csv('WOA18/woa18_all_n'+str(pt)+'an01.csv', header=[1]).reset_index().drop('index', axis=1).rename(columns={'#COMMA SEPARATED LATITUDE':'lat',' LONGITUDE':'lon', ' AND VALUES AT DEPTHS (M):0':0}).set_index(['lat','lon']).loc[:,0]
    lon = data.index.get_level_values('lon')
    lat = data.index.get_level_values('lat')
    data = data.unstack()
    time = pd.to_datetime({'year': [2000],'month': [pt],'day': [1]})
    
    #-----------------------------------------------------------------
    # case 1: match pixels to high resolution satellite data
    if lon[1]-lon[0] >= grid: #i.e. if upsampling to finer grid size
        # Regrid data and interpolate though:
        # First find the indices to index by lat/lon
        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)

        # Restrict the data to the specified lat/lons
        data_indexed = data.loc[min_lat:max_lat, min_lon:max_lon]

        # Create data matrix with coordinates
        data_mat = pd.DataFrame(data_indexed)
        # data_mat.columns = lon[min_lon:max_lon]
        # data_mat.index = lat[min_lat:max_lat]
        #sort the data so that columns are ascending (matches 'new_shape' below)
        data_mat = data_mat.reindex(sorted(data_mat.columns), axis=1) # sort columns in ascending order

        # Create a new matrix with new gridded coordinates
        lat_new = np.arange(min_lat, max_lat+grid, grid)
        lon_new = np.arange(min_lon, max_lon+grid, grid)
        new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
        new_shape.columns=lon_new
        new_shape.index=lat_new

        # Now reindex data to new coordinates - add in NaNs to interpolate through
        # important: find corresponding nearest value in new index/columns to replace data coords by
        new_idx = [new_shape.index[abs(new_shape.index.values-i).argmin()] for i in data_mat.index]
        new_cols = [new_shape.columns[abs(new_shape.columns.values-i).argmin()] for i in data_mat.columns]
        # now rename idx/cols with new values in the data matrix (this allows the reindex_like function to properly map and insert nans below)
        data_mat.set_axis(new_idx, axis=0, inplace=True)
        data_mat.set_axis(new_cols, axis=1, inplace=True)

        data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
        data_proc = data_proc.rename(columns={0:'data'})
        data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
        data_proc['datetime'] = np.tile(pt,data_proc.shape[0])
        data_proc.reset_index(inplace=True)
        data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
        # data_proc.reset_index(inplace=True)
        
    #-----------------------------------------------------------------
    # case 2: downsample to courser grid
    if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
        # Bin the data
        
        # First find the indices to index by lat/lon
        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
        
        # Restrict the data to the NE Pacific lat/lons
        data_indexed = data.loc[min_lat:max_lat, min_lon:max_lon]
        
        # Convert time/lats/lons into repeating matrices to match data dimensions
        lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
        lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
        time_mat = np.tile(pt, len(np.ravel(lat_mat)))
        
        # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
        # Need these as pandas dataframes to use binning scheme:
        d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
        data_long = pd.DataFrame(data=d)
        
        # Bin data as averages across gridded spatial bins:
        to_bin = lambda x: np.round(x / grid) * grid
        data_long['latbins'] = data_long.lat.map(to_bin)
        data_long['lonbins'] = data_long.lon.map(to_bin)
        data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
        
        # Rename binned columns + drop mean lat/lons:
        data_proc = data_proc.drop(columns=['lat', 'lon'])
    # Save to output array
    if n == 0: # for first iteration, define pre-allocated variable in dict as new binned data
        SSN = data_proc
    else: # for subsequent iterations, add to existing dict variable by "stacking" new binned datasets row-wise
        SSN = pd.concat([SSN,data_proc]).groupby(['datetime','latbins','lonbins']).mean()
    print(str(n+1)+' of '+str(np.size(months))+' iterations completed!')

#------------------------------------------------------------------------------
#### Silicate
#------------------------------------------------------------------------------
# Pre-allocate dask array to store new binned data to
Si = dd.from_array(da.empty((1,len(months))))

for n,pt in enumerate(months):
    # Load up wind data file (Copernicus timeseries data is downloaded into a single netCDF)
    if pt<10:
      data = pd.read_csv('WOA18/woa18_all_i0'+str(pt)+'an01.csv', header=[1]).reset_index().drop('index', axis=1).rename(columns={'#COMMA SEPARATED LATITUDE':'lat',' LONGITUDE':'lon', ' AND VALUES AT DEPTHS (M):0':0}).set_index(['lat','lon']).loc[:,0]
    else:
      data = pd.read_csv('WOA18/woa18_all_i'+str(pt)+'an01.csv', header=[1]).reset_index().drop('index', axis=1).rename(columns={'#COMMA SEPARATED LATITUDE':'lat',' LONGITUDE':'lon', ' AND VALUES AT DEPTHS (M):0':0}).set_index(['lat','lon']).loc[:,0]
    lon = data.index.get_level_values('lon')
    lat = data.index.get_level_values('lat')
    data = data.unstack()
    time = pd.to_datetime({'year': [2000],'month': [pt],'day': [1]})
    
    #-----------------------------------------------------------------
    # case 1: match pixels to high resolution satellite data
    if lon[1]-lon[0] >= grid: #i.e. if upsampling to finer grid size
        # Regrid data and interpolate though:
        # First find the indices to index by lat/lon
        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)

        # Restrict the data to the specified lat/lons
        data_indexed = data.loc[min_lat:max_lat, min_lon:max_lon]

        # Create data matrix with coordinates
        data_mat = pd.DataFrame(data_indexed)
        # data_mat.columns = lon[min_lon:max_lon]
        # data_mat.index = lat[min_lat:max_lat]
        #sort the data so that columns are ascending (matches 'new_shape' below)
        data_mat = data_mat.reindex(sorted(data_mat.columns), axis=1) # sort columns in ascending order

        # Create a new matrix with new gridded coordinates
        lat_new = np.arange(min_lat, max_lat+grid, grid)
        lon_new = np.arange(min_lon, max_lon+grid, grid)
        new_shape = pd.DataFrame(np.ones((lat_new.shape[0],lon_new.shape[0])))
        new_shape.columns=lon_new
        new_shape.index=lat_new

        # Now reindex data to new coordinates - add in NaNs to interpolate through
        # important: find corresponding nearest value in new index/columns to replace data coords by
        new_idx = [new_shape.index[abs(new_shape.index.values-i).argmin()] for i in data_mat.index]
        new_cols = [new_shape.columns[abs(new_shape.columns.values-i).argmin()] for i in data_mat.columns]
        # now rename idx/cols with new values in the data matrix (this allows the reindex_like function to properly map and insert nans below)
        data_mat.set_axis(new_idx, axis=0, inplace=True)
        data_mat.set_axis(new_cols, axis=1, inplace=True)

        data_proc = pd.DataFrame(data_mat.reindex_like(new_shape).stack(dropna=False))
        data_proc = data_proc.rename(columns={0:'data'})
        data_proc.index = data_proc.index.set_names(['latbins','lonbins'])
        data_proc['datetime'] = np.tile(pt,data_proc.shape[0])
        data_proc.reset_index(inplace=True)
        data_proc.set_index(['datetime','latbins','lonbins'], inplace=True)
        # data_proc.reset_index(inplace=True)
        
    #-----------------------------------------------------------------
    # case 2: downsample to courser grid
    if lon[1]-lon[0] < grid: # i.e. if interpolating to courser grid
        # Bin the data
        
        # First find the indices to index by lat/lon
        latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
        loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)
        
        # Restrict the data to the NE Pacific lat/lons
        data_indexed = data.loc[min_lat:max_lat, min_lon:max_lon]
        
        # Convert time/lats/lons into repeating matrices to match data dimensions
        lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
        lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row
        time_mat = np.tile(pt, len(np.ravel(lat_mat)))
        
        # Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
        # Need these as pandas dataframes to using binning scheme:
        d = {'datetime':time_mat,'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
        data_long = pd.DataFrame(data=d)
        
        # Bin data as averages across gridded spatial bins:
        to_bin = lambda x: np.round(x / grid) * grid
        data_long['latbins'] = data_long.lat.map(to_bin)
        data_long['lonbins'] = data_long.lon.map(to_bin)
        data_proc = data_long.groupby(['datetime', 'latbins', 'lonbins']).mean()
        
        # Rename binned columns + drop mean lat/lons:
        data_proc = data_proc.drop(columns=['lat', 'lon'])
    # Save to output array
    if n == 0: # for first iteration, define pre-allocated variable in dict as new binned data
        Si = data_proc
    else: # for subsequent iterations, add to existing dict variable by "stacking" new binned datasets row-wise
        Si = pd.concat([Si,data_proc]).groupby(['datetime','latbins','lonbins']).mean()
    print(str(n+1)+' of '+str(np.size(months))+' iterations completed!')

#===============================================================================
# write data to file
from google.colab import files
SSN.to_csv(write_to+'SSN'+'_'+str(round(grid,3))+'_deg.csv')
Si.to_csv(write_to+'Si'+'_'+str(round(grid,3))+'_deg.csv')
files.download(write_to+'SSN'+'_'+str(round(grid,3))+'_deg.csv')
files.download(write_to+'Si'+'_'+str(round(grid,3))+'_deg.csv')


Processing WOA18 SSN data...

Loading data...
1 of 7 iterations completed!
2 of 7 iterations completed!
3 of 7 iterations completed!
4 of 7 iterations completed!
5 of 7 iterations completed!
6 of 7 iterations completed!
7 of 7 iterations completed!
1 of 7 iterations completed!
2 of 7 iterations completed!
3 of 7 iterations completed!
4 of 7 iterations completed!
5 of 7 iterations completed!
6 of 7 iterations completed!
7 of 7 iterations completed!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#%% ETOPO2 data
# This bathymetry data is used to mask out land pixels when interpolating
vars_ = xr.open_dataset('drive/MyDrive/etopo2.nc')
data = vars_.btdata.values
lat = vars_.coords['lat'].values
lon = vars_.coords['lon'].values
#-----------------------------------------------------------------
# Bin the data

# First find the indices to index by lat/lon
latinds = np.argwhere((lat >= min_lat) & (lat <= max_lat)).astype(int)
loninds = np.argwhere((lon >= min_lon) & (lon <= max_lon)).astype(int)

# Restrict the data to the NE Pacific lat/lons
data_indexed = data[latinds[:,None], loninds[None,:]][:,:,0]

# Convert time/lats/lons into repeating matrices to match data dimensions
lat_mat = np.tile(lat[latinds], len(lon[loninds])) # latitude is repeated by column
lon_mat = np.tile(lon[loninds], len(lat[latinds])).T # transpose to repeat longitude by row

# Now create a new matrix with the unravel matrices into vectors - np.ravel does this row-by-row
# Need these as pandas dataframes to using binning scheme:
d = {'lat':np.ravel(lat_mat),'lon':np.ravel(lon_mat),'data':np.ravel(data_indexed)}
data_long = pd.DataFrame(data=d)

# Bin data as averages across gridded spatial bins:
to_bin = lambda x: np.round(x / grid) * grid
data_long['latbins'] = data_long.lat.map(to_bin)
data_long['lonbins'] = data_long.lon.map(to_bin)
data_proc = data_long.groupby(['latbins', 'lonbins']).mean()

#-----------------------------------------------------------------
# Rename binned columns + drop mean lat/lons:
data_proc = data_proc.drop(columns=['lat', 'lon'])
data_proc.reset_index(inplace=True) # remove index specification on columns
# etopo = data_proc.iloc[:,1:]
etopo = data_proc
etopo.set_index(['lonbins','latbins'], inplace=True)
# del data_proc, vars_, data_long
#===============================================================================
# Write data to file
from google.colab import files
etopo.to_csv('SO_etopo'+'_'+str(round(grid,3))+'_deg.csv')
files.download('SO_etopo'+'_'+str(round(grid,3))+'_deg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>